In [1]:
from hyperpyyaml import load_hyperpyyaml
import os
from cosyvoice.cli.frontend import CosyVoiceFrontEnd
from cosyvoice.cli.model import CosyVoiceModel, CosyVoice2Model, CosyVoice3Model

hyper_yaml_path = '/home/ecs-user/code/happen/A100-04/happen/AUDIO/HydraVox/pretrained_models/Fun-CosyVoice3-0.5B/cosyvoice3_mtp.yaml'
model_dir = "/home/ecs-user/code/happen/A100-04/happen/AUDIO/HydraVox/pretrained_models/Fun-CosyVoice3-0.5B"

failed to import ttsfrd, use wetext instead


/home/ecs-user/anaconda3/envs/hydravox/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
with open(hyper_yaml_path, 'r') as f:
    configs = load_hyperpyyaml(f, overrides={'qwen_pretrain_path': os.path.join(model_dir, 'CosyVoice-BlankEN')})

2025-12-24 10:17:39,406 INFO input frame rate=25
/home/ecs-user/anaconda3/envs/hydravox/lib/python3.11/site-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


In [3]:
sample_rate = configs['sample_rate']

In [4]:
frontend = CosyVoiceFrontEnd(configs['get_tokenizer'],
                                    configs['feat_extractor'],
                                    '{}/campplus.onnx'.format(model_dir),
                                    '{}/speech_tokenizer_v3.onnx'.format(model_dir),
                                    '{}/spk2info.pt'.format(model_dir),
                                    configs['allowed_special'])

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
2025-12-24 10:17:42.238347138 [W:onnxruntime:, transformer_memcpy.cc:74 ApplyImpl] 14 Memcpy nodes are added to the graph main_graph for CUDAExecutionProvider. It might have negative impact on performance (including unable to run CUDA graph). Set session_options.log_severity_level=1 to see the detail logs before this message.
2025-12-24 10:17:42.243573347 [W:onnxruntime:, session_state.cc:1166 VerifyEachNodeIsAssignedToAnEp] Some nodes were not assigned to the preferred execution providers which may or may not have an negative impact on performance. e.g. ORT explicitly assigns shape related ops to CPU to improve perf.
2025-12-24 10:17:42.243581939 [W:onnxruntime:, session_state.cc:1168 VerifyEachNodeIsAssignedToAnEp] Rerunning with verbose output on a no

2025-12-24 10:17:43,945 DEBUG https://www.modelscope.cn:443 "GET /api/v1/models/pengzhendong/wetext/revisions HTTP/1.1" 200 222
2025-12-24 10:17:44,066 DEBUG https://www.modelscope.cn:443 "GET /api/v1/models/pengzhendong/wetext/repo/files?Revision=master&Recursive=True HTTP/1.1" 200 None
2025-12-24 10:17:44,160 DEBUG Starting new HTTPS connection (1): www.modelscope.cn:443
2025-12-24 10:17:44,356 DEBUG https://www.modelscope.cn:443 "GET /api/v1/models/pengzhendong/wetext HTTP/1.1" 200 None
2025-12-24 10:17:44,470 DEBUG https://www.modelscope.cn:443 "GET /api/v1/models/pengzhendong/wetext/revisions HTTP/1.1" 200 222


2025-12-24 10:17:44,663 DEBUG https://www.modelscope.cn:443 "GET /api/v1/models/pengzhendong/wetext/repo/files?Revision=master&Recursive=True HTTP/1.1" 200 None


In [5]:
model = CosyVoice3Model(configs['llm'], configs['flow'], configs['hift'], True)

In [6]:
model.load('{}/llm_multihead_100k.pt'.format(model_dir),
                '{}/flow.pt'.format(model_dir),
                '{}/hift.pt'.format(model_dir))

In [23]:
prompt_text = frontend.text_normalize("同学们，今天的课堂从一个简单的问题开始", split=False, text_frontend=True)
model_input = frontend.frontend_zero_shot("[y][vè][y][vè]，你好", prompt_text, "/home/ecs-user/code/happen/A100-04/happen/AUDIO/HydraVox/outputs/emotion_batch/hanxue_prompt.wav", sample_rate, '')
output= model.tts(**model_input, stream=False, speed=1.0)

In [24]:
from IPython.display import Audio
Audios = []
for item in output:
    Audios.append(item['tts_speech'])

In [25]:
Audios

[tensor([[ 2.0303e-04,  1.1462e-04,  6.8975e-05,  ..., -9.8811e-04,
          -6.8489e-04, -6.5607e-04]])]

In [26]:
import numpy as np
Audio(Audios[0].cpu().numpy(), rate=sample_rate)